# Huawei Research France

## Introduction

Le réseau d'accès optique (OAN) est une solution courante de réseau d'accès domestique à large bande dans le monde entier. Il relie les abonnés des terminaux à leur fournisseur de services. Les défaillances du réseau affectent à la fois la qualité du service (QoS) et l'expérience de l'utilisateur (la qualité d'expérience QoE). Pour réduire les dommages, il est important de prévoir à l'avance les défaillances du réseau et de les réparer à temps. Les algorithmes d'apprentissage machine (ML) ont été largement utilisés comme solution pour construire ces modèles de prédiction des pannes. 

Cependant, la plupart des modèles d'apprentissage automatique sont spécifiques aux données et ont tendance à se dégrader lorsque la distribution des données change. Le premier défi de données de Huawei France de cette année vise à résoudre ce problème. 

Vous recevrez un ensemble de données étiquetées sur le réseau d'accès optique d'une ville que nous appelons "A" (que nous appelons le domaine source) et un ensemble de données pour la plupart non étiquetées d'une ville "B" (que nous appelons le domaine cible).

On vous demande de construire une solution d'apprentissage par transfert en utilisant les données sources étiquetées et les données cibles non étiquetées pour entraîner un modèle de prédiction de panne pour la ville B. Il s'agit d'un **problème d'adaptation de domaine non supervisée (UDA)**. Pour être précis, nous incluons un petit nombre de points cibles étiquetés dans l'ensemble d'entraînement, de sorte que nous pouvons appeler cette configuration "UDA à quelques coups" ou "adaptation de domaine semi-supervisée".


1. **valeurs manquantes** : il y a beaucoup de valeurs manquantes dans les données ;
2. **séries temporelles de données de capteurs** ;
3. **déséquilibre des classes** : les défaillances du réseau sont rares, il s'agit donc d'un problème de classification très déséquilibré. 

## Contexte

Les technologies de transmission ont évolué pour intégrer les technologies optiques jusque dans les réseaux d'accès, au plus près de l'abonné. Actuellement, la fibre optique est le support de transmission par excellence en raison de sa capacité à propager le signal sur de longues distances sans régénération, de sa faible latence et de sa très grande largeur de bande. La fibre optique, initialement déployée dans les réseaux à très longue distance et à très haut débit, tend aujourd'hui à se généraliser pour offrir des services plus grand public en termes de bande passante. Il s'agit des technologies FTTH pour "Fiber to the Home ".

Le FTTH généralement adopté par les opérateurs est une architecture PON (Passive Optical Network). Le PON est une architecture point à multipoint basée sur les éléments suivants :
- Une infrastructure de fibre optique partagée. L'utilisation de coupleurs optiques dans le réseau est la base de l'architecture et de l'ingénierie de déploiement. Les coupleurs sont utilisés pour desservir plusieurs zones ou plusieurs abonnés.


- Equipement central faisant office de terminaison de ligne optique (OLT). L'OLT gère la diffusion et la réception des flux à travers les interfaces du réseau. Il reçoit les signaux des abonnés et diffuse un contenu basé sur des services spécifiques. 


- Équipements terminaux :
    - ONT (Optical Network Terminations) dans le cas où l'équipement est dédié à un client et que la fibre atteint le client. Il s'agit alors d'une architecture de type FTTH (Fiber To The Home). Il n'y a qu'une seule fibre par client (les signaux sont bidirectionnels).
    - ONU (Optical Network Unit) dans le cas où l'équipement est dédié à un bâtiment entier. Il s'agit alors d'une architecture de type FTTB (Fiber To The Building).

    
<img src="https://image.makewebeasy.net/makeweb/0/p4Ky6EVg4/optical%20fiber-knowledge/Apps_FTTx_Fig3.png">

Les données pour ce défi sont collectées à partir de capteurs au niveau de l'ONT.

### Les données

Les données proviennent de deux villes différentes : la ville A (la source) et la ville B (la cible). Les données sont étiquetées pour la ville A mais (principalement) non étiquetées pour la ville B (seulement 20% des données étiquetées sont connues pour la ville B). Pour les deux villes A et B, les données sont une série temporelle collectée pendant environ 60 jours. La granularité de la série temporelle est de 15 minutes. Les échantillons représentent différents utilisateurs (donc différents ONT). A chaque pas de temps, nous disposons d'une mesure en dix dimensions des caractéristiques suivantes (entre parenthèses, les unités de chaque caractéristique).

- **current** : courant de polarisation du module optique de l'ONT GPON (mA)
- **err_down_bip** : nombre de trames descendantes ONT avec erreur BIP (entier)
- **err_up_bip** : nombre de trames ONT amont avec erreur BIP (entier)
- **olt_recv** : puissance de réception du module optique GPON ONT de l'ONU (dBm)
- **rdown** : débit descendant de l'ONT GPON (Mbs)
- **recv** : puissance de réception du module optique GPON ONT (dBm)
- **rup** : débit amont de l'ONT GPON (Mbs)
- **send** : puissance d'émission du module optique GPON ONT (dBm)
- **temp** : température du module optique GPON ONT (Celsius)
- **volt** : tension d'alimentation du module optique GPON ONT (mV)
- **étiquettes** : 0 (faible) ou 1 (échec) pour l'échantillon. 

L'objectif du défi est de séparer le faible de l'échec, les bonnes données sont juste données comme information secondaire (pouvant être utilisées pour la calibration), ainsi l'objectif est de soumettre un classificateur binaire.

Soit $x_t$ l'échantillon collecté au jour $t$, alors l'étiquette correspondante est calculée au jour $t+7$. Notre objectif est de prédire un échec à partir de données provenant de 7 jours auparavant.  


Les données sont données avec la forme **[users, timestamps, features]** et les features sont données dans le même ordre que celui présenté ci-dessus. Pour chaque utilisateur et chaque horodatage, nous agrégeons sept jours de données.

Notez que l'ensemble de données publiques (qui vous est remis avec le kit de démarrage) et l'ensemble de données privées (utilisé pour évaluer vos soumissions sur le serveur) proviennent de la même distribution, donc en principe vous pourriez utiliser les données cibles publiques étiquetées pour apprendre un classificateur et soumettre la fonction réelle. Cela irait à l'encontre de l'objectif de l'apprentissage par transfert, nous avons donc décidé de transformer légèrement mais significativement l'ensemble de données privées pour rendre cette stratégie non performante.

In [100]:
import pandas as pd

In [138]:
sample = 20

test = pd.DataFrame(X_train.source[sample], columns=["current", 
                                          "err_down_bip", 
                                          "err_up_bip", 
                                          "olt_recv", 
                                          "rdown", 
                                          "recv", 
                                          "rup",
                                          "send", "temp", "volt"])
test.head()

,current,err_down_bip,err_up_bip,olt_recv,rdown,recv,rup,send,temp,volt
0,13.0,563.0,25.0,NaN,0.003,-30.459999,0.001,2.32,45.0,3300.0
1,13.0,541.0,48.0,NaN,0.001,-30.459999,0.001,2.25,45.0,3300.0
2,13.0,587.0,68.0,NaN,0.001,-30.459999,0.001,2.35,45.0,3300.0
3,13.0,641.0,79.0,NaN,0.001,-30.459999,0.001,2.23,45.0,3300.0
4,13.0,585.0,96.0,NaN,0.000,-30.459999,0.001,2.19,45.0,3300.0


In [139]:
test.describe()

,current,err_down_bip,err_up_bip,olt_recv,rdown,recv,rup,send,temp,volt
count,672.000000,672.000000,672.000000,0.0,672.000000,672.000000,672.000000,672.00000,672.000000,672.000000
mean,12.660714,733.159241,54.818451,NaN,0.576213,-30.460159,0.010720,2.26110,44.453869,3299.970215
std,0.473820,208.409286,86.458672,NaN,1.941489,0.000160,0.026105,0.05637,0.701891,0.771516
min,12.000000,412.000000,0.000000,NaN,0.000000,-30.459999,0.000000,2.16000,43.000000,3280.000000
25%,12.000000,631.000000,9.000000,NaN,0.001000,-30.459999,0.001000,2.22000,44.000000,3300.000000
50%,13.000000,753.000000,27.000000,NaN,0.001000,-30.459999,0.001000,2.26000,44.000000,3300.000000
75%,13.000000,825.250000,65.250000,NaN,0.102500,-30.459999,0.007000,2.31000,45.000000,3300.000000
max,13.000000,4649.000000,597.000000,NaN,20.476000,-30.459999,0.317000,2.36000,46.000000,3300.000000


#### Données manquantes

Vous remarquerez que certaines données sont manquantes dans les ensembles de données. Il peut y avoir plusieurs raisons :

1. Aucune donnée n'a été collectée à une date spécifique pour un utilisateur spécifique.
2. Le processus de collecte des données ne parvient pas à récupérer une caractéristique.
    
Cela fait partie du défi de surmonter cette difficulté de la vie réelle.

### Métriques

- Accuracy (**acc**): Le nombre d'étiquettes correctement prédites par rapport au nombre total d'échantillons.  [sklearn function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score). 
- Area unther the ROC curve (**auc**). Ce score nous donne la probabilité qu'une instance d'échec soit mieux notée qu'une instance faible par la fonction discriminante binaire [sklearn function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html).
- Average precision (**ap**): il résume une courbe précision-rappel sous la forme de la moyenne pondérée des précisions obtenues à chaque seuil, l'augmentation du rappel par rapport au seuil précédent étant utilisée comme poids [sklearn function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score).
- **Precision@Recall**: est un score hybride implémenté dans `utils.scores`. Il calcule la précision lorsque le rappel est à un certain pourcentage, c'est-à-dire, recall est la précision lorsque le rappel est à k%.

**NOTE : Average precision (ap) est la métrique officiel d'évaluation**.

## Getting started


Pour installer `ramp-workflow`:
```
pip install git+https://github.com/paris-saclay-cds/ramp-workflow.git
```

Cette commande installera la bibliothèque `rampwf` et le script `ramp-test` que vous pouvez utiliser pour vérifier votre soumission avant de la soumettre. Vous n'avez pas besoin de connaître ce paquetage pour participer au défi, mais il pourrait être utile de jeter un coup d'œil à la [documentation](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/index.html) si vous souhaitez savoir ce qui se passe lorsque nous testons votre modèle, en particulier la page [exécution RAMP](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/scoring.html) pour comprendre `ramp-test`, et les [commandes](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/command_line.html) pour comprendre les différentes options de la ligne de commande. 

In [128]:
import numpy as np
import rampwf as rw

In [129]:
problem = rw.utils.assert_read_problem()

### Les données

Exécutez le script `prepare_data.py` dans `./data`. Notez que les données publiques qui vous sont données sont différentes des données privées utilisées pour évaluer vos soumissions sur le serveur.

Les données d'apprentissage sont composées de données source et cible provenant respectivement de la ville A et de la ville B. 

Dans la vie réelle, le problème FTTH comporte trois classes : 

- 1) le débit est normal et tout se passe bien (bon), 
- 2) le débit est faible mais la connexion fonctionne toujours (faible), 
- 3) défaillance. 

Pour la détection des défaillances de l'OAN, nous sommes intéressés par une classification binaire entre les deux classes : **[faible, échec]**.

Vous êtes libre d'exploiter les données de la bonne classe, mais lors de la notation, vous n'êtes jugé que sur la classification binaire.

**Les données sources**
- X_train.source : Les données pour les classes faible et défaillante.
- X_train.source_bkg : Données pour la classe bonne.
- y_train.source : Étiquettes pour X_train.source, 0 : faible et 1 : échec.
   
**Les données cibles**
 - X_train.target : Données cibles (étiquetées) pour les classes faible et échec.
 - X_train.target_unlabeled : Données cibles non étiquetées.
 - X_train.target_bkg : Données cibles pour la classe bonne.
 - y_train.target : Étiquettes pour X_train.target, 0 : faible et 1 : échec.

Puisque nous nous intéressons à la performance du classificateur sur les données cibles, l'ensemble de test est composé entièrement de données cibles. predict recevra à la fois X_test.target et X_test.target_bkg, et on s'attend à ce qu'il produise des probabilités des étiquettes faible et échec uniquement pour X_test.target.

In [130]:
X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

Train data
Optical Dataset composed of
46110 source samples
50862 source background samples
438 target labeled samples
8202 target unlabeled samples
29592 target background samples
 Optical Dataset labels composed of
46110 labels of source samples
438 labels of target samples

Test data
Optical Dataset composed of
0 source samples
0 source background samples
17758 target labeled samples
0 target unlabeled samples
47275 target background samples
 Optical Dataset labels composed of
0 labels of source samples
17758 labels of target samples



Les données d'entrée sont tridimensionnelles (échantillon, temps, caractéristiques). Le temps a 672 dimensions (4 fois une heure $\times$ 24 heures $\times$ 7 jours). Il contient des valeurs nan, il doit donc être nettoyé.

In [131]:
X_train.source[6].shape

(672, 10)

### Classification

Vous devez soumettre un extracteur de caractéristiques et un classificateur. La fonction transform de l'extracteur de caractéristiques est exécutée sur chaque donnée d'entrée (cible, source, bkg) et les tableaux résultants sont passés aux fonctions fit et predict du classificateur. L'extracteur de caractéristiques du kit de départ remplace nans par zéro, et aplatit la matrice en **(sample, 6720)**.

In [42]:
%%writefile submissions/source_rf/feature_extractor.py
import numpy as np

class FeatureExtractor:

    def __init__(self):
        pass

    def transform(self, X):
        # Deal with NaNs inplace
        np.nan_to_num(X, copy=False)
        # We flatten the input, originally 3D (sample, time, dim) to
        # 2D (sample, time * dim)
        X = X.reshape(X.shape[0], -1)
        return X


Overwriting submissions/source_rf/feature_extractor.py


The starting kit implements a naive domain adaptation where the model (random forest) trained on the source is used to classify the target.

In [43]:
%%writefile submissions/source_rf/classifier.py
from sklearn.ensemble import RandomForestClassifier
from utils.dataset import OpticalDataset, OpticalLabels
from lightgbm import LGBMClassifier

import numpy as np

class Classifier:

    def __init__(self):
        self.clf = LGBMClassifier(
            n_estimators=50, 
            max_depth=20, 
            random_state=44, 
            num_leaves=31,
            n_jobs=-1)
        print(self.clf)

    def fit(self, X_source, X_source_bkg, X_target, X_target_unlabeled,
            X_target_bkg, y_source, y_target):
        self.clf.fit(X_source, y_source)

    def predict_proba(self, X_target, X_target_bkg):
        y_proba = self.clf.predict_proba(X_target)
        return y_proba


Overwriting submissions/source_rf/classifier.py


Vous pouvez regarder le code du flux de travail à `external_imports/utils/workflow.py` pour voir exactement comment vos soumissions sont chargées et utilisées. Vous pouvez exécuter l'entraînement et la prédiction de votre soumission ici dans le notebook. Lorsque vous exécutez `ramp-test`, nous faisons une validation croisée ; ici vous utilisez les données complètes de formation pour former et les données de test pour tester. [Cette page](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/scoring.html) vous donne un bref aperçu de ce qui se passe en coulisses lorsque vous exécutez le script `ramp-test`.

In [67]:
X_train

Optical Dataset composed of
46110 source samples
50862 source background samples
438 target labeled samples
8202 target unlabeled samples
29592 target background samples

In [44]:
trained_workflow = problem.workflow.train_submission('submissions/source_rf', X_train, y_train)
y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)

LGBMClassifier(max_depth=20, n_estimators=50, random_state=44)
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


### The scores

Nous calculons six scores sur la classification. Tous les scores sont implémentés dans `external_imports.utils.scores.py` donc vous pouvez regarder les définitions précises là. Le score officiel de la compétition est ap.

In [45]:
ap    = problem.score_types[0]
rec5  = problem.score_types[1]
rec10 = problem.score_types[2]
rec20 = problem.score_types[3]
acc   = problem.score_types[4]
auc   = problem.score_types[5]

In [46]:
print('ap test score    = {}'.format(ap(y_test.target, y_test_pred[:,1])))
print('rec5 test score  = {}'.format(rec5(y_test.target, y_test_pred[:,1])))
print('rec10 test score = {}'.format(rec10(y_test.target, y_test_pred[:,1])))
print('rec20 test score = {}'.format(rec20(y_test.target, y_test_pred[:,1])))
print('acc test score   = {}'.format(acc(y_test.target, y_test_pred.argmax(axis=1))))
print('auc test score   = {}'.format(auc(y_test.target, y_test_pred[:,1])))

ap test score    = 0.1838770011572445
rec5 test score  = 0.08674804121255875
rec10 test score = 0.16564951837062836
rec20 test score = 0.32083696126937866
acc test score   = 0.823234598490821
auc test score   = 0.6293792968994895


### The cross validation scheme

Nous utilisons une validation croisée dix fois (stratifiée lorsque les étiquettes sont disponibles) pour tous les ensembles de données. Dans chaque split, 20% des instances sont dans l'ensemble de validation, à l'exception des données cibles étiquetées qui servent principalement à la validation (pour obtenir une estimation non biaisée des scores de test, évalués entièrement sur des échantillons cibles étiquetés). Nous plaçons vingt points cibles étiquetés dans les splits d'entraînement. La raison en est que lorsque nous étendons nos services à large bande à la ville B, nous pouvons obtenir rapidement un petit ensemble de données étiquetées, mais nous aimerions déployer notre détecteur de défaillance sans attendre deux mois pour recueillir des données comparables à celles de la ville A.

Le schéma de validation croisée (voir `problem.get_cv`) est implémenté dans la classe `TLShuffleSplit` de `external_imports.utils.cv.py`, si vous voulez y regarder de plus près.

Vous êtes libre de jouer avec la coupure train/test et la validation croisée lors du développement de vos modèles mais sachez que nous utiliserons la même configuration sur le serveur officiel que celle du kit RAMP (sur un ensemble différent de quatre campagnes qui ne sera pas disponible pour vous).

La cellule suivante passe par les mêmes étapes que le script d'évaluation officiel (`ramp-test`).

In [69]:
splits = problem.get_cv(X_train, y_train)

In [13]:
splits = problem.get_cv(X_train, y_train)

y_test_preds = []
for fold_i, (train_is, valid_is) in enumerate(splits):
    trained_workflow = problem.workflow.train_submission(
        'submissions/starting_kit', X_train, y_train, train_is)
    X_fold_train = X_train.slice(train_is)
    X_fold_valid = X_train.slice(valid_is)
    
    y_train_pred = problem.workflow.test_submission(trained_workflow, X_fold_train)
    y_valid_pred = problem.workflow.test_submission(trained_workflow, X_fold_valid)
    y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)
    print('-------------------------------------')
    print('training ap on fold {} = {}'.format(
        fold_i, ap(y_train.slice(train_is).target, y_train_pred[:,1])))
    print('validation ap on fold {} = {}'.format(
        fold_i, ap(y_train.slice(valid_is).target, y_valid_pred[:,1])))
    print('test ap on fold {} = {}'.format(fold_i, ap(y_test.target, y_test_pred[:,1])))
    
    y_test_preds.append(y_test_pred)

-------------------------------------
training ap on fold 0 = 0.30833333333333335
validation ap on fold 0 = 0.2637875964895809
test ap on fold 0 = 0.16218430339780684
-------------------------------------
training ap on fold 1 = 0.21250000000000002
validation ap on fold 1 = 0.2555942077788053
test ap on fold 1 = 0.16361016472786805
-------------------------------------
training ap on fold 2 = 0.2
validation ap on fold 2 = 0.29440601825201235
test ap on fold 2 = 0.1745388926023523
-------------------------------------
training ap on fold 3 = 0.7375
validation ap on fold 3 = 0.28218715512682335
test ap on fold 3 = 0.16904411795376056
-------------------------------------
training ap on fold 4 = 0.21250000000000002
validation ap on fold 4 = 0.24879604051634688
test ap on fold 4 = 0.16172210972525408


KeyboardInterrupt: 

Nous calculons à la fois le score moyen du test et le score de la mise en sac de vos dix modèles. Le classement officiel sera déterminé par le score de test mis en sac (sur des ensembles de données différents de ceux dont vous disposez). Votre score public sera le score de validation mis en sac (le calcul de la moyenne est [légèrement plus compliqué](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/rampwf/utils/combine.py#L56) car nous devons nous occuper correctement des masques de validation croisée). 

In [16]:
bagged_y_pred = np.array(y_test_preds).mean(axis=0)
print('Mean ap score = {}'.format(
    np.mean([ap(y_test.target, y_test_pred[:,1]) for y_test_pred in y_test_preds])))
print('Bagged ap score = {}'.format(
    ap(y_test.target, np.array([y_test_pred for y_test_pred in y_test_preds]).mean(axis=0)[:,1])))

Mean ap score = 0.1662199176814084
Bagged ap score = 0.1688256992369087


## Exemple submissions

Outre le kit de départ, nous vous proposons deux autres exemples de soumissions. L'extracteur de caractéristiques est le même dans les trois. `source_rf` est similaire au kit de départ, mais utilise des arbres plus nombreux et plus profonds, pour obtenir un meilleur score. `target_rf` est une autre soumission extrême qui utilise seulement l'instance d'entraînement de la cible (peu) étiquetée pour apprendre un classificateur. Il a une performance légèrement moins bonne que `source_rf` ce qui signifie que les données sources améliorent le classificateur même si les distributions sources et cibles sont différentes.

### Resultats:
|          | ap             | rec-5         | rec-10         | rec-20         | acc            |  auc           | 
|:---------|:--------------:|:-------------:|:--------------:|:--------------:|:--------------:|:--------------:|   
|source_rf | 0.191 ± 0.0026 | 0.073 ± 0.002 | 0.176 ± 0.0032 | 0.357 ± 0.0075 | 0.84 ± 0.0014  | 0.637 ± 0.0063 | 
|target_rf | 0.163 ± 0.0218 | 0.067 ± 0.0182| 0.138 ± 0.0339 | 0.272 ± 0.0537 | 0.813 ± 0.036  | 0.591 ± 0.0399 | 

La grande question de l'apprentissage par transfert à résoudre est la suivante : **Comment combiner les données cibles à faible biais et à haute variance avec les données sources à faible biais et à haute variance**. D'autres questions auxquelles nous nous attendons à voir des réponses :

1. Peut-on faire un meilleur prétraitement (amputation des données manquantes, utilisation du temps d'une manière plus intelligente) dans l'extracteur de caractéristiques ?
2. Normalement, les données d'arrière-plan (bonnes instances) ne participent pas au scoring, mais elles peuvent informer le classifieur du changement de distribution. Comment utiliser au mieux cette information ?

## Local testing (before submission)

You submission will contain a `feature_extractor.py` implementing a FeatureExtractor class with a `transform` function (no `fit`) and a `classifier.py` implementing a Classifier class with a `fit` and `predict_proba` functions as in the starting kit. You should place it in the `submission/<submission_name>` folder in your RAMP kit folder. To test your submission, go to your RAMP kit folder in the terminal and type
```
ramp-test --submission <submission_name>
```
It will train and test your submission much like we did it above in this notebook, and print the foldwise and summary scores. You can try it also in this notebook:

In [ ]:
!ramp-test --submission target_rf

If you want to have a local leaderboard, use the `--save-output` option when running `ramp-test`, then try `ramp-show leaderboard` with different options. For example:
```
ramp-show leaderboard --mean --metric "['ap','auc']" --step "['valid','test']" --precision 3
```
and
```
ramp-show leaderboard --bagged --metric "['auc']"
```

RAMP also has an experimental hyperopt feature, with random grid search implemented. If you want to use it, type
```
ramp-hyperopt --help
```
and check out the example submission [here](https://github.com/ramp-kits/titanic/tree/hyperopt/submissions/starting_kit_h).

## Contact

You can contact the organizers in the Slack of the challenge, join by [clicking here](https://join.slack.com/t/huaweiramp/shared_invite/zt-qbf4vy9s-0NS4~V898h40x8cI2KHEfQ). 